<a href="https://colab.research.google.com/github/GaoangLiu/ipynb/blob/master/NaiveBayes_Classify_Insincere_Quora_Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load packages 
import math
import re
import os
import timeit
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import logging

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
logging.basicConfig(format='[%(asctime)s %(levelname)-8s] %(message)s', level=logging.INFO, datefmt='%m-%d %H:%M:%S')


In [0]:
# Get data
! wget -O quora.zip ali.140714.xyz:8000/quora.zip 
! unzip quora.zip 
! ls 

In [0]:
class Bayes():
  def __init__(self):
    self.train = None
    self.test = None 

  def load_data(self, train_file='train.csv', test_file='test.csv'):
      """ Load train, test csv files and return pandas.DataFrame
      """
      self.train = pd.read_csv(train_file, engine='python', encoding='utf-8', error_bad_lines=False)
      self.test = pd.read_csv(test_file, engine='python', encoding='utf-8', error_bad_lines=False)
      logging.info('CSV data loaded')
  
  def countvectorize(self):
      tv = TfidfVectorizer(ngram_range=(1,3), token_pattern=r'\w{1,}',
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1, max_features=5000)
      # cv = CountVectorizer()
      tv.fit(self.train.question_text)
      self.vector_train = tv.transform(self.train.question_text)
      self.vector_test  = tv.transform(self.test.question_text)
      logging.info("Train & test text tokenized")

  def run_model(self):
      nb = MultinomialNB()
      labels = self.train.target
      x_train, x_val, y_train, y_val = train_test_split(self.vector_train, labels, test_size=0.07, random_state=23)
      nb.fit(x_train, y_train)
      y_preds = nb.predict(x_val)
      logging.info(f"Accuracy score: {accuracy_score(y_val, y_preds)}")

      y_preds = nb.predict(self.vector_test)
      return y_preds
  
  def save_predictions(self, y_preds):
      sub = pd.read_csv('sample_submission.csv')
      sub['prediction'] = y_preds 
      sub.to_csv('submission.csv', index=False)
      logging.info('Prediction exported to submisison.csv')

b = Bayes()
b.load_data()
b.countvectorize()
yp = b.run_model()


In [0]:
set(dups)